# [Day 16 - Ticket Translation](https://adventofcode.com/2020/day/16)
## Part 1

In [1]:
import parse

input = open("inputs/16-input.txt").read()

rules, ticket, tickets = input.split("\n\n")

# Parse rules
rule_dict = {}
rules = rules.split("\n")
p = parse.compile("{field}: {a:d}-{b:d} or {c:d}-{d:d}")
for r in rules:
    rule = p.parse(r)
    rule_dict[rule["field"]] = list(range(rule["a"],rule["b"]+1)) + list(range(rule["c"],rule["d"]+1))

# My ticket values
my_ticket = list(map(int, ticket.split("\n")[1].split(",")))

# Nearby ticket values
tickets = [list(map(int, t.split(','))) for t in tickets.split("\n")[1:-1]]

In [2]:
import numpy as np
from itertools import compress

def part1(rules:dict, tickets:list):
    errors = []
    for t in tickets:
        invalid = [all([num not in vals for vals in rules.values()]) for num in t]
        errors += list(compress(t, invalid))
    return sum(errors)

part1(rule_dict,tickets)

29851

## Part 2

In [3]:
# Valid tickets are my ticket + others where all values obey at least 1 of the rules
valid_tickets = [my_ticket] + [t for t in tickets if all([any([num in vals for vals in rule_dict.values()]) for num in t])]

# Transpose tickets to list of all ticket values for each field
valid_columns = list(map(list, zip(*valid_tickets)))

# For each column, which fields are possible matches (set comprehension within dict comprehension :heart_eyes:)
possibles = {i:{rule for rule in rule_dict.keys() if set(rule_dict[rule]).issuperset(set(col))} for i,col in enumerate(valid_columns)}

# Column -> field dict
solution = {}

# Check for columns with unique possible -> add to solution -> remove from other columns
def iterate(possibles, solution={}):
    for i in possibles:
        if len(possibles[i])==1:
            solution[i] = possibles[i].pop()
            for j in possibles:
                possibles[j].discard(solution[i])
    return possibles, solution

# Repeat until solution is complete
while len(solution) < len(valid_columns):
    possibles, solution = iterate(possibles,solution)

In [4]:
from math import prod

# Product of my ticket values for fields starting "departure"
prod([my_ticket[k] for k in solution if solution[k].startswith("departure")])

3029180675981